## Plan

We will build a complete Hugging Face Fine-Tuning Pipeline for binary classification.

Data Bridge: Convert your Pandas DataFrame into a Hugging Face Dataset.

Tokenization: Apply the BioBERT tokenizer to the txt_criteria column (identified in your audit as the complex text source).

Metrics: Define a compute_metrics function using F1-score and Precision (crucial for imbalanced data).

Training: Set up TrainingArguments optimized for a quick MVP (fewer epochs, lower batch size).

## The Libraries

In [1]:
import pandas as pd
import numpy as np
!pip install torch
!pip install datasets
from datasets import Dataset
from transformers import (AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from sklearn.model_selection import train_test_split

!{sys.executable} -m pip install --upgrade accelerate
!{sys.executable} -m pip install --upgrade "transformers[torch]"
import torch

import os
import csv

/home/ruima/.pyenv/versions/clintrialpredict/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


zsh:1: parse error near `-m'
zsh:1: parse error near `-m'


## The Dataset

In [5]:
DATA_PATH = "/home/ruima/code/delaunan/clintrialpredict/data"
df = pd.read_csv(os.path.join(DATA_PATH, 'project_data.csv'))
df.columns
#df.shape
#df.head

Index(['nct_id', 'start_date_type', 'start_date', 'study_type',
       'overall_status', 'phase', 'number_of_arms', 'why_stopped', 'target',
       'start_year', 'phase_ordinal', 'covid_exposure', 'includes_us',
       'is_international', 'agency_class', 'allocation', 'intervention_model',
       'primary_purpose', 'masking', 'gender', 'healthy_volunteers', 'adult',
       'child', 'older_adult', 'num_primary_endpoints', 'best_pathology',
       'therapeutic_area', 'therapeutic_subgroup_name', 'competition_broad',
       'competition_niche', 'txt_tags', 'txt_criteria'],
      dtype='object')

In [11]:
#df['overall_status'].value_counts()
print(df['overall_status'].unique())
df.shape

['COMPLETED' 'TERMINATED' 'WITHDRAWN' 'SUSPENDED']


(105336, 32)

## Setup and Data Preparation

In [7]:
# ==========================================
# 1. SETUP & DATA PREPARATION
# ==========================================
# We filter for the target classes and map them to 0/1
# 0 = Failed (Risk), 1 = Completed (Success)
# ==========================================


# Filter for relevant statuses only
target_mask = df['overall_status'].isin(['COMPLETED', 'TERMINATED',
                                         'WITHDRAWN', 'SUSPENDED'])
df_clean = df[target_mask].copy()

# Create Binary Target: 1 if Failed, 0 if Completed
df_clean['label'] = df_clean['overall_status'].apply(
    lambda x: 1 if x == 'Completed' else 0
)

# Handle Missing Text (Crucial for BERT)
df_clean['txt_criteria'] = df_clean['txt_criteria'].fillna("missing")

# Split Data (Stratified because of class imbalance)
train_df, val_df = train_test_split(
    df_clean, test_size=0.2, stratify=df_clean['label'], random_state=42
)

# Convert to Hugging Face Dataset objects
train_dataset = Dataset.from_pandas(train_df[['txt_criteria', 'label']])
val_dataset = Dataset.from_pandas(val_df[['txt_criteria', 'label']])

## Tokenization

In [8]:
import sys
!{sys.executable} -m pip install "transformers==4.39.3" --force-reinstall


  Using cached transformers-4.39.3-py3-none-any.whl.metadata (134 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
  Using cached numpy-2.2.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached pyyaml-6.0.3-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (2.4 kB)
  Using cached regex-2025.11.3-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (40 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tokenizers-0.15.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached safetensors-0.7.0-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached fsspec-2

In [9]:
# ===========================
# PATCH: Disable chat template loading
# ===========================
#from huggingface_hub import file_download

# Monkey-patch the internal endpoint call that tries to fetch chat templates
#file_download._request_with_retry = lambda *args, **kwargs: None

# ===========================
# Load BioBERT tokenizer
# ===========================
#from transformers import AutoTokenizer

# ==========================================
# 2. TOKENIZATION
# ==========================================

model_name = "dmis-lab/biobert-v1.1"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=False,             # <--- CRITICAL FIX
    trust_remote_code=True,
    revision="main")

def tokenize_function(examples):
    return tokenizer(
        examples["txt_criteria"],
        padding="max_length",
        truncation=True,
        max_length=512 # BioBERT limit
    )

# Apply tokenization
print("Tokenizing datasets...")
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)

Tokenizing datasets...


Map: 100%|██████████| 21068/21068 [03:45<00:00, 93.62 examples/s] 


## Metrics Definition

In [10]:
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score

# ==========================================
# 3. METRICS DEFINITION (Forced Print)
# ==========================================
def compute_metrics(eval_pred):
    # Unpack the tuple
    logits, labels = eval_pred

    # 1. Convert Logits to Probabilities
    # Check if logits is a tuple (some models return extra data), take the first element if so
    if isinstance(logits, tuple):
        logits = logits[0]

    # Convert to tensor -> softmax -> numpy
    # We select column [:, 1] for the probability of the Positive Class (1)
    probs = torch.nn.functional.softmax(torch.tensor(logits), dim=-1)[:, 1].numpy()

    # 2. Get Predictions (0 or 1)
    predictions = np.argmax(logits, axis=-1)

    # 3. Calculate Metrics
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='binary'
    )
    acc = accuracy_score(labels, predictions)
    auc = roc_auc_score(labels, probs)

    # 4. FORCE PRINT (flush=True is key here)
    print("\n" + "="*40, flush=True)
    print(f"📊 EVALUATION REPORT (Step/Epoch End)", flush=True)
    print("="*40, flush=True)
    print(f"  • Accuracy:   {acc:.4f}", flush=True)
    print(f"  • F1 Score:   {f1:.4f}", flush=True)
    print(f"  • Precision:  {precision:.4f}", flush=True)
    print(f"  • Recall:     {recall:.4f}", flush=True)
    print(f"  • AUC:        {auc:.4f}", flush=True)
    print("="*40 + "\n", flush=True)

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'auc': auc
    }

## Model and Training Arguments

In [15]:
!{sys.executable} -m pip install --upgrade accelerate
!{sys.executable} -m pip install --upgrade "transformers[torch]"

  Using cached transformers-4.57.3-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
Using cached transformers-4.57.3-py3-none-any.whl (12.0 MB)
Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers━━━━━━━━━━━━ 0/2 [tokenizers]
    Found existing installation: transformers 4.39.32m0/2 [tokenizers]
    Uninstalling transformers-4.39.3:╺━━━━━━━━━━━━━━━━━━━ 1/2 [transformers]
      Successfully uninstalled transformers-4.39.3━━━━━━━━━━━━━━━━ 1/2 [transformers]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [transformers] [transformers]


In [18]:
# ==========================================
# 4. MODEL & TRAINING ARGUMENTS (FIXED)
# ==========================================
# Load Model with Classification Head
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
)

# Define Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",          # <--- CHANGED: evaluation_strategy -> eval_strategy
    save_strategy="epoch",          # Save model at end of every epoch
    learning_rate=2e-5,             # Standard BERT learning rate
    per_device_train_batch_size=8,  # Keep small to avoid GPU OOM (8 or 16)
    per_device_eval_batch_size=16,
    num_train_epochs=2,             # Start small for MVP (1-2 epochs)
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",     # Optimize for F1, not Accuracy
    logging_steps=50,
    fp16=torch.cuda.is_available(), # Use Mixed Precision if GPU available (Speedup)
    report_to="none"                # Disable WandB/MLFlow for simple local run
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# Evaluate the model
print("Starting Evaluation...")
metrics = trainer.evaluate()

# Print metrics
for k, v in metrics.items():
    if isinstance(v, float):
        print(f"{k}: {v:.4f}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_609958/1874185466.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Starting Evaluation...



📊 EVALUATION REPORT (Step/Epoch End)
  • Accuracy:   0.1236
  • F1 Score:   0.0000
  • Precision:  0.0000
  • Recall:     0.0000
  • AUC:        nan

eval_loss: 0.8567
eval_model_preparation_time: 0.0086
eval_accuracy: 0.1236
eval_f1: 0.0000
eval_precision: 0.0000
eval_recall: 0.0000
eval_auc: nan
eval_runtime: 9454.4273
eval_samples_per_second: 2.2400
eval_steps_per_second: 0.1400


/home/ruima/.pyenv/versions/clintrialpredict/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/ruima/.pyenv/versions/clintrialpredict/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


## Execution

In [ ]:
# ==========================================
# 5. EXECUTION
# ==========================================
# Warning: This line starts the heavy compute
print("Starting Fine-Tuning...")
trainer.train()

# Save the final model for your Streamlit app
trainer.save_model("./final_biobert_risk_model")